# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [15]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    sentences = source_text.split('\n')
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in sentences]
    sentences = target_text.split('\n')
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] + [target_vocab_to_int['<EOS>']] for sentence in sentences]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/pasgille/Workbench/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [19]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [20]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [21]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    # Dropout
    drop_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, keep_prob)
    
    enc_output, enc_state = tf.nn.dynamic_rnn(drop_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [22]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the training process. Used by BasicDecoder to read inputs.
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)


    # Basic decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 

    # Perform dynamic decoding using the decoder
    basic_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_summary_length)[0]
    return basic_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [24]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the inference process.
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                            start_tokens,
                                                            end_of_sequence_id)

    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                    inference_helper,
                                                    encoder_state,
                                                    output_layer)

    # Perform dynamic decoding using the decoder
    basic_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                        impute_finished=True,
                                                        maximum_iterations=max_target_sequence_length)[0]
    return basic_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [26]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):
        train_basic_decoder_ouput = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, 
                             output_layer, keep_prob)
    
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        infer_basic_decoder_ouput = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                             target_vocab_to_int['<EOS>'], max_target_sequence_length,
                             target_vocab_size, output_layer, batch_size, keep_prob)
    return train_basic_decoder_ouput, infer_basic_decoder_ouput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [31]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers,
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, 
                                                                       target_sequence_length, 
                                                                       max_target_sentence_length, 
                                                                       rnn_size,num_layers, 
                                                                       target_vocab_to_int, 
                                                                       target_vocab_size, 
                                                                       batch_size, 
                                                                       keep_prob, dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [32]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 5


### Build the Graph
Build the graph using the neural network you implemented.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/1077 - Train Accuracy: 0.2605, Validation Accuracy: 0.3050, Loss: 5.7878
Epoch   0 Batch   10/1077 - Train Accuracy: 0.1982, Validation Accuracy: 0.3050, Loss: 5.6175
Epoch   0 Batch   15/1077 - Train Accuracy: 0.2355, Validation Accuracy: 0.3050, Loss: 5.1604
Epoch   0 Batch   20/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 4.7385
Epoch   0 Batch   25/1077 - Train Accuracy: 0.2434, Validation Accuracy: 0.3079, Loss: 4.3663
Epoch   0 Batch   30/1077 - Train Accuracy: 0.2277, Validation Accuracy: 0.3050, Loss: 4.0668
Epoch   0 Batch   35/1077 - Train Accuracy: 0.2340, Validation Accuracy: 0.3050, Loss: 3.8470
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2434, Validation Accuracy: 0.3185, Loss: 3.7019
Epoch   0 Batch   45/1077 - Train Accuracy: 0.2629, Validation Accuracy: 0.3349, Loss: 3.6106
Epoch   0 Batch   50/1077 - Train Accuracy: 0.2590, Validation Accuracy: 0.3356, Loss: 3.5877
Epoch   0 Batch   55/1077 - Train Accuracy: 0.2910, Validati

Epoch   0 Batch  445/1077 - Train Accuracy: 0.3968, Validation Accuracy: 0.4737, Loss: 2.5966
Epoch   0 Batch  450/1077 - Train Accuracy: 0.4203, Validation Accuracy: 0.4702, Loss: 2.4720
Epoch   0 Batch  455/1077 - Train Accuracy: 0.4748, Validation Accuracy: 0.4801, Loss: 2.2986
Epoch   0 Batch  460/1077 - Train Accuracy: 0.4047, Validation Accuracy: 0.4709, Loss: 2.5048
Epoch   0 Batch  465/1077 - Train Accuracy: 0.3635, Validation Accuracy: 0.4659, Loss: 2.6541
Epoch   0 Batch  470/1077 - Train Accuracy: 0.4013, Validation Accuracy: 0.4702, Loss: 2.5145
Epoch   0 Batch  475/1077 - Train Accuracy: 0.4383, Validation Accuracy: 0.4670, Loss: 2.3089
Epoch   0 Batch  480/1077 - Train Accuracy: 0.4071, Validation Accuracy: 0.4702, Loss: 2.4751
Epoch   0 Batch  485/1077 - Train Accuracy: 0.4230, Validation Accuracy: 0.4794, Loss: 2.4269
Epoch   0 Batch  490/1077 - Train Accuracy: 0.4098, Validation Accuracy: 0.4773, Loss: 2.4513
Epoch   0 Batch  495/1077 - Train Accuracy: 0.4160, Validati

Epoch   0 Batch  885/1077 - Train Accuracy: 0.4759, Validation Accuracy: 0.4847, Loss: 1.6398
Epoch   0 Batch  890/1077 - Train Accuracy: 0.4851, Validation Accuracy: 0.4847, Loss: 1.6600
Epoch   0 Batch  895/1077 - Train Accuracy: 0.4086, Validation Accuracy: 0.4925, Loss: 1.7917
Epoch   0 Batch  900/1077 - Train Accuracy: 0.4332, Validation Accuracy: 0.4862, Loss: 1.8643
Epoch   0 Batch  905/1077 - Train Accuracy: 0.4234, Validation Accuracy: 0.4961, Loss: 1.7653
Epoch   0 Batch  910/1077 - Train Accuracy: 0.4583, Validation Accuracy: 0.4851, Loss: 1.7036
Epoch   0 Batch  915/1077 - Train Accuracy: 0.3803, Validation Accuracy: 0.4911, Loss: 1.9160
Epoch   0 Batch  920/1077 - Train Accuracy: 0.4383, Validation Accuracy: 0.4897, Loss: 1.7627
Epoch   0 Batch  925/1077 - Train Accuracy: 0.4323, Validation Accuracy: 0.4858, Loss: 1.7077
Epoch   0 Batch  930/1077 - Train Accuracy: 0.4605, Validation Accuracy: 0.4954, Loss: 1.7182
Epoch   0 Batch  935/1077 - Train Accuracy: 0.4352, Validati

Epoch   1 Batch  250/1077 - Train Accuracy: 0.4801, Validation Accuracy: 0.5405, Loss: 1.1816
Epoch   1 Batch  255/1077 - Train Accuracy: 0.4723, Validation Accuracy: 0.5252, Loss: 1.2733
Epoch   1 Batch  260/1077 - Train Accuracy: 0.4981, Validation Accuracy: 0.5327, Loss: 1.1959
Epoch   1 Batch  265/1077 - Train Accuracy: 0.4898, Validation Accuracy: 0.5330, Loss: 1.2179
Epoch   1 Batch  270/1077 - Train Accuracy: 0.4285, Validation Accuracy: 0.5380, Loss: 1.3095
Epoch   1 Batch  275/1077 - Train Accuracy: 0.4825, Validation Accuracy: 0.5160, Loss: 1.1743
Epoch   1 Batch  280/1077 - Train Accuracy: 0.4727, Validation Accuracy: 0.5266, Loss: 1.2488
Epoch   1 Batch  285/1077 - Train Accuracy: 0.5130, Validation Accuracy: 0.5337, Loss: 1.1615
Epoch   1 Batch  290/1077 - Train Accuracy: 0.4730, Validation Accuracy: 0.5199, Loss: 1.2452
Epoch   1 Batch  295/1077 - Train Accuracy: 0.4511, Validation Accuracy: 0.5224, Loss: 1.2976
Epoch   1 Batch  300/1077 - Train Accuracy: 0.4441, Validati

Epoch   1 Batch  690/1077 - Train Accuracy: 0.4680, Validation Accuracy: 0.4986, Loss: 1.0023
Epoch   1 Batch  695/1077 - Train Accuracy: 0.4625, Validation Accuracy: 0.5117, Loss: 0.9738
Epoch   1 Batch  700/1077 - Train Accuracy: 0.4621, Validation Accuracy: 0.5124, Loss: 0.9693
Epoch   1 Batch  705/1077 - Train Accuracy: 0.4613, Validation Accuracy: 0.5149, Loss: 1.0575
Epoch   1 Batch  710/1077 - Train Accuracy: 0.4289, Validation Accuracy: 0.5067, Loss: 0.9984
Epoch   1 Batch  715/1077 - Train Accuracy: 0.4098, Validation Accuracy: 0.4975, Loss: 1.0123
Epoch   1 Batch  720/1077 - Train Accuracy: 0.4219, Validation Accuracy: 0.5078, Loss: 1.0515
Epoch   1 Batch  725/1077 - Train Accuracy: 0.4654, Validation Accuracy: 0.4989, Loss: 0.9372
Epoch   1 Batch  730/1077 - Train Accuracy: 0.4551, Validation Accuracy: 0.5021, Loss: 0.9879
Epoch   1 Batch  735/1077 - Train Accuracy: 0.4660, Validation Accuracy: 0.5320, Loss: 0.9911
Epoch   1 Batch  740/1077 - Train Accuracy: 0.5051, Validati

...

Epoch  58 Batch  135/1077 - Train Accuracy: 0.9231, Validation Accuracy: 0.9265, Loss: 0.0574
Epoch  58 Batch  140/1077 - Train Accuracy: 0.9630, Validation Accuracy: 0.9304, Loss: 0.0534
Epoch  58 Batch  145/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9258, Loss: 0.0580
Epoch  58 Batch  150/1077 - Train Accuracy: 0.9182, Validation Accuracy: 0.9077, Loss: 0.0538
Epoch  58 Batch  155/1077 - Train Accuracy: 0.9430, Validation Accuracy: 0.9283, Loss: 0.0472
Epoch  58 Batch  160/1077 - Train Accuracy: 0.9426, Validation Accuracy: 0.9190, Loss: 0.0534
Epoch  58 Batch  165/1077 - Train Accuracy: 0.9262, Validation Accuracy: 0.8995, Loss: 0.0514
Epoch  58 Batch  170/1077 - Train Accuracy: 0.9008, Validation Accuracy: 0.9208, Loss: 0.0611
Epoch  58 Batch  175/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9109, Loss: 0.0596
Epoch  58 Batch  180/1077 - Train Accuracy: 0.9371, Validation Accuracy: 0.9244, Loss: 0.0544
Epoch  58 Batch  185/1077 - Train Accuracy: 0.9215, Validati

Epoch  58 Batch  575/1077 - Train Accuracy: 0.9501, Validation Accuracy: 0.9151, Loss: 0.0456
Epoch  58 Batch  580/1077 - Train Accuracy: 0.9412, Validation Accuracy: 0.9286, Loss: 0.0447
Epoch  58 Batch  585/1077 - Train Accuracy: 0.9349, Validation Accuracy: 0.9244, Loss: 0.0473
Epoch  58 Batch  590/1077 - Train Accuracy: 0.8853, Validation Accuracy: 0.9112, Loss: 0.0511
Epoch  58 Batch  595/1077 - Train Accuracy: 0.9410, Validation Accuracy: 0.9279, Loss: 0.0452
Epoch  58 Batch  600/1077 - Train Accuracy: 0.9342, Validation Accuracy: 0.9197, Loss: 0.0457
Epoch  58 Batch  605/1077 - Train Accuracy: 0.9523, Validation Accuracy: 0.9389, Loss: 0.0557
Epoch  58 Batch  610/1077 - Train Accuracy: 0.9021, Validation Accuracy: 0.9272, Loss: 0.0486
Epoch  58 Batch  615/1077 - Train Accuracy: 0.9437, Validation Accuracy: 0.9315, Loss: 0.0574
Epoch  58 Batch  620/1077 - Train Accuracy: 0.9742, Validation Accuracy: 0.9244, Loss: 0.0488
Epoch  58 Batch  625/1077 - Train Accuracy: 0.9379, Validati

Epoch  58 Batch 1015/1077 - Train Accuracy: 0.9160, Validation Accuracy: 0.9354, Loss: 0.0561
Epoch  58 Batch 1020/1077 - Train Accuracy: 0.9332, Validation Accuracy: 0.9293, Loss: 0.0532
Epoch  58 Batch 1025/1077 - Train Accuracy: 0.9275, Validation Accuracy: 0.9098, Loss: 0.0477
Epoch  58 Batch 1030/1077 - Train Accuracy: 0.9473, Validation Accuracy: 0.9247, Loss: 0.0550
Epoch  58 Batch 1035/1077 - Train Accuracy: 0.9401, Validation Accuracy: 0.9315, Loss: 0.0384
Epoch  58 Batch 1040/1077 - Train Accuracy: 0.9428, Validation Accuracy: 0.9098, Loss: 0.0506
Epoch  58 Batch 1045/1077 - Train Accuracy: 0.9066, Validation Accuracy: 0.9283, Loss: 0.0597
Epoch  58 Batch 1050/1077 - Train Accuracy: 0.9488, Validation Accuracy: 0.9279, Loss: 0.0423
Epoch  58 Batch 1055/1077 - Train Accuracy: 0.9355, Validation Accuracy: 0.9237, Loss: 0.0665
Epoch  58 Batch 1060/1077 - Train Accuracy: 0.9477, Validation Accuracy: 0.9215, Loss: 0.0437
Epoch  58 Batch 1065/1077 - Train Accuracy: 0.9230, Validati

Epoch  59 Batch  380/1077 - Train Accuracy: 0.9539, Validation Accuracy: 0.9247, Loss: 0.0495
Epoch  59 Batch  385/1077 - Train Accuracy: 0.9289, Validation Accuracy: 0.9297, Loss: 0.0497
Epoch  59 Batch  390/1077 - Train Accuracy: 0.8992, Validation Accuracy: 0.9226, Loss: 0.0678
Epoch  59 Batch  395/1077 - Train Accuracy: 0.9487, Validation Accuracy: 0.9357, Loss: 0.0486
Epoch  59 Batch  400/1077 - Train Accuracy: 0.9254, Validation Accuracy: 0.9247, Loss: 0.0522
Epoch  59 Batch  405/1077 - Train Accuracy: 0.9227, Validation Accuracy: 0.9276, Loss: 0.0596
Epoch  59 Batch  410/1077 - Train Accuracy: 0.8812, Validation Accuracy: 0.9226, Loss: 0.0625
Epoch  59 Batch  415/1077 - Train Accuracy: 0.9211, Validation Accuracy: 0.9261, Loss: 0.0462
Epoch  59 Batch  420/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9237, Loss: 0.0455
Epoch  59 Batch  425/1077 - Train Accuracy: 0.9040, Validation Accuracy: 0.9279, Loss: 0.0407
Epoch  59 Batch  430/1077 - Train Accuracy: 0.9270, Validati

Epoch  59 Batch  820/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9116, Loss: 0.0529
Epoch  59 Batch  825/1077 - Train Accuracy: 0.9395, Validation Accuracy: 0.9311, Loss: 0.0404
Epoch  59 Batch  830/1077 - Train Accuracy: 0.8809, Validation Accuracy: 0.9222, Loss: 0.0615
Epoch  59 Batch  835/1077 - Train Accuracy: 0.9250, Validation Accuracy: 0.9162, Loss: 0.0477
Epoch  59 Batch  840/1077 - Train Accuracy: 0.9316, Validation Accuracy: 0.9336, Loss: 0.0576
Epoch  59 Batch  845/1077 - Train Accuracy: 0.9305, Validation Accuracy: 0.9165, Loss: 0.0514
Epoch  59 Batch  850/1077 - Train Accuracy: 0.9129, Validation Accuracy: 0.9318, Loss: 0.0618
Epoch  59 Batch  855/1077 - Train Accuracy: 0.9160, Validation Accuracy: 0.9304, Loss: 0.0567
Epoch  59 Batch  860/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9197, Loss: 0.0570
Epoch  59 Batch  865/1077 - Train Accuracy: 0.9386, Validation Accuracy: 0.9329, Loss: 0.0474
Epoch  59 Batch  870/1077 - Train Accuracy: 0.9190, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [38]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [42]:
# translate_sentence = 'he saw a old yellow truck .'
translate_sentence = 'new jersey is sometimes quiet during autumn .'



"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [113, 12, 208, 144, 20, 180, 148, 60]
  English Words: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', '.']

Prediction
  Word Ids:      [214, 200, 36, 172, 194, 328, 316, 304, 36, 34, 353, 289, 1]
  French Words: l' inde est froid en octobre , mais est à sec . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.